# Taobao DSSM Training Pipeline

This notebook implements the complete training pipeline for the DSSM recommendation model.

In [10]:
# Environment Setup and Data Loading

import pandas as pd
import numpy as np
import os
import sys
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Add src to path for imports
sys.path.append('/kaggle/input/dssm-project-code')

# Import modules
from src import config
from src import data_processing
from src import feature_engineering
from src import model

# Set data path
DATA_PATH = f'../input/{config.DATASET_FOLDER_NAME}/'

logger.info("Environment and modules imported successfully!")
logger.info(f"Configuration loaded: BATCH_SIZE={config.BATCH_SIZE}, LEARNING_RATE={config.LEARNING_RATE}")

## Data Preprocessing

In [11]:
# Data Preprocessing

if not os.path.exists(os.path.join(config.OUTPUT_DIR, 'train_data.csv')):
    logger.info("Starting data preprocessing pipeline...")
    data_processing.run_pipeline(
        data_path=DATA_PATH,
        output_dir=config.OUTPUT_DIR,
        chunk_size=config.CHUNKSIZE,
        negative_ratio=config.NEG_POS_RATIO
    )
else:
    logger.info("Data files already exist, skipping preprocessing.")

## Feature Engineering

In [12]:
# Feature Engineering

logger.info("Loading data...")
train_df = pd.read_csv(os.path.join(config.OUTPUT_DIR, 'train_data.csv'))
test_df = pd.read_csv(os.path.join(config.OUTPUT_DIR, 'test_data.csv'))

logger.info(f"Training data shape: {train_df.shape}")
logger.info(f"Test data shape: {test_df.shape}")

# Perform feature engineering
train_df, test_df = feature_engineering.perform_feature_engineering(
    train_df, 
    test_df, 
    config.SPARSE_FEATURES, 
    config.DENSE_FEATURES, 
    config.OUTPUT_DIR
)

logger.info("Feature engineering completed!")
logger.info(f"Processed training data shape: {train_df.shape}")

--- 开始进行特征编码 ---


Encoding Sparse Features: 100%|██████████| 15/15 [02:22<00:00,  9.53s/it]



稀疏特征Label Encoding完成。
稠密特征MinMax Scaling完成。

特征处理器已保存到 /kaggle/working/
--- 特征工程完毕！ ---


## Model Training

In [13]:
# Model Training

# 1. Create model input dictionaries
train_model_input = {name: train_df[name] for name in config.ALL_FEATURES}
train_label = train_df['label'].values
test_model_input = {name: test_df[name] for name in config.ALL_FEATURES}
test_label = test_df['label'].values

logger.info(f"Training samples: {len(train_label)}")
logger.info(f"Test samples: {len(test_label)}")
logger.info(f"Positive samples in training: {train_df['label'].sum()}")
logger.info(f"Positive samples in test: {test_df['label'].sum()}")

In [14]:
# 2. Create feature columns
with open(os.path.join(config.OUTPUT_DIR, 'feature_encoders.pkl'), 'rb') as f:
    feature_encoders = pickle.load(f)

# User feature columns
user_feature_columns = [
    model.SparseFeat(
        name=feat, 
        vocabulary_size=len(feature_encoders[feat].classes_) + 1, 
        embedding_dim=config.EMBEDDING_DIM
    )
    for feat in config.USER_SPARSE_FEATURES
]

from typing import List, Union
# Item feature columns
item_feature_columns: List[Union[model.SparseFeat, model.DenseFeat]] = [
    model.SparseFeat(
        name=feat, 
        vocabulary_size=len(feature_encoders[feat].classes_) + 1, 
        embedding_dim=config.EMBEDDING_DIM
    )
    for feat in config.ITEM_SPARSE_FEATURES
]

# Add dense features
item_feature_columns.append(model.DenseFeat(name='price', dimension=1))

logger.info(f"User feature columns: {len(user_feature_columns)}")
logger.info(f"Item feature columns: {len(item_feature_columns)}")

In [15]:
# 3. Build and compile model
dssm_model = model.DSSM(
    user_feature_columns, 
    item_feature_columns, 
    dnn_units=config.DNN_UNITS, 
    temp=config.TEMP
)

dssm_model.summary()

dssm_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.LEARNING_RATE), 
    loss="binary_crossentropy", 
    metrics=[tf.keras.metrics.AUC(name='auc')]
)

logger.info("Model compiled successfully!")

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ adgroup_id (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cate_id (InputLayer)      │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ campaign_id (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ customer_id (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ brand (InputLayer)        │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pid (InputLayer)          │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_id (InputLayer)      │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cms_segid (InputLayer)    │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cms_group_id (InputLayer) │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ final_gender_code         │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ age_level (InputLayer)    │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pvalue_level (InputLayer) │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shopping_level            │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ occupation (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ new_user_class_level      │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ emb_adgroup_id            │ (None, 1, 16)          │      9,275,408 │ adgroup_id[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ emb_cate_id (Embedding)   │ (None, 1, 16)          │        100,032 │ cate_id[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ emb_campaign_id      

 Total params: 34,906,738 (133.16 MB)

 Trainable params: 34,906,738 (133.16 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# 4. Train model
early_stopping = EarlyStopping(
    monitor='val_auc', 
    mode='max', 
    patience=2, 
    verbose=1, 
    restore_best_weights=True
)

logger.info("Starting model training...")

history = dssm_model.fit(
    train_model_input, 
    train_label, 
    batch_size=config.BATCH_SIZE,
    epochs=config.EPOCHS,
    validation_data=(test_model_input, test_label),
    callbacks=[early_stopping]
)

logger.info("Model training completed!")

Epoch 1/10
1493/1493 ━━━━━━━━━━━━━━━━━━━━ 43s 19ms/step - auc: 0.5187 - loss: 0.5220 - val_auc: 0.6244 - val_loss: 0.2505
Epoch 2/10
1493/1493 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - auc: 0.7102 - loss: 0.4570 - val_auc: 0.6278 - val_loss: 0.2608
Epoch 3/10
1493/1493 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - auc: 0.7766 - loss: 0.4223 - val_auc: 0.6215 - val_loss: 0.2773
Epoch 4/10
1493/1493 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - auc: 0.8042 - loss: 0.4036 - val_auc: 0.6143 - val_loss: 0.2961
Epoch 4: early stopping
Restoring model weights from the end of the best epoch: 2.


## Model Evaluation and Saving

In [17]:
# Model Evaluation

# Evaluate on test set
test_loss, test_auc = dssm_model.evaluate(test_model_input, test_label)
logger.info(f"Test Loss: {test_loss:.4f}")
logger.info(f"Test AUC: {test_auc:.4f}")

# Save model
model_save_path = os.path.join(config.OUTPUT_DIR, 'dssm_model')
dssm_model.save(model_save_path)
logger.info(f"Model saved to: {model_save_path}")

# Save training history
history_save_path = os.path.join(config.OUTPUT_DIR, 'training_history.pkl')
if history is not None:
    with open(history_save_path, 'wb') as f:
        pickle.dump(history.history, f)
    logger.info(f"Training history saved to: {history_save_path}")
else:
    logger.warning("No training history to save")

89016/89016 ━━━━━━━━━━━━━━━━━━━━ 243s 3ms/step - auc: 0.6256 - loss: 0.2579


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=/kaggle/working/dssm_model.